In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'pod_compatibility'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Coffee, Tea, & Espresso Makers"""

regex_pattern=r'(?i)(\d+.\d*.Inch.+x.?\d+.?Inch)|(\d+.?\d+\/\d+.?Inch.?x.?\d+.?\d+\/\d+.?Inch)|(\d+.?\d+\/\d+\-Inch)|(\d+\.?\d*\s?.?Inch)|()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/top_width.sql', params=params)
df

attribute                     value    customers_name  \
0    pod_compatibility                       n/a  bedbathandbeyond   
1    pod_compatibility                       n/a  bedbathandbeyond   
2    pod_compatibility                       n/a  bedbathandbeyond   
3    pod_compatibility                       n/a  bedbathandbeyond   
4    pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond   
..                 ...                       ...               ...   
670  pod_compatibility                       n/a  bedbathandbeyond   
671  pod_compatibility                       n/a  bedbathandbeyond   
672  pod_compatibility                       n/a  bedbathandbeyond   
673  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond   
674  pod_compatibility                       n/a  bedbathandbeyond   

    external_id  \
0      68642839   
1      46946423   
2      67673377   
3      66450375   
4      66326403   
..          ...   
670    64803784   
671    64803807   
672    68287252   
673    66326045   
674    17429019   

                                                                     long_desc  \
0    Zwilling® J.A. Henckels Sorrento Plus 27 oz. French Press in Clear/Silver   
1                                Zojirushi Fresh Brew Plus 12-Cup Coffee Maker   
2                       Zojirushi™ 10-Cup Thermal Carafe Coffee Maker in Black   
3                    Zaasenhaus 34 oz. Stainless Steel Hot & Cold Brew Infuser   
4                           West Virginia University Single Serve Coffee Maker   
..                                                                         ...   
670                    ASOBU 32 oz. Pour Over Insulated Coffee Maker in Copper   
671                     ASOBU 32 oz. Pour Over Insulated Coffee Maker in Black   
672                               Aroma® 6-Cup Programable Tea Maker in Pastel   
673                         Arizona State University Single Serve Coffee Maker   
674                                         AeroPress® Coffee & Espresso Maker   

    resolution  
0     standard  
1     standard  
2     standard  
3     standard  
4     standard  
..         ...  
670   standard  
671   standard  
672   standard  
673   standard  
674   standard  

[675 rows x 6 columns]

In [5]:
df[df['external_id'].astype(str)=='46517203']

Empty DataFrame
Columns: [attribute, value, customers_name, external_id, long_desc, resolution]
Index: []

In [6]:
# df['value'][13000:14000].to_list()

In [7]:
df['value'].explode().value_counts()

n/a                                                                  545
Single-Serve Coffee Pods                                              70
["Single-Serve Coffee Capsules","Single-Serve Espresso Capsules"]     45
Single-Serve Coffee Cups                                               5
Single-Serve Espresso Capsules                                         5
Single-Serve Espresso Pods                                             2
["Single-Serve Espresso Capsules","Single-Serve Espresso Pods"]        1
["Single-Serve Coffee Cups","Single-Serve Coffee Pods"]                1
Single-Serve Coffee Capsules                                           1
Name: value, dtype: int64

In [8]:
# metric='in'
# range_string= """
# """

# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# list_values

# kwp=KeywordProcessor()
# print(kwp.add_keywords_from_list(list_values))
# # Extract Key Words
# df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['matches'].explode().value_counts()

In [9]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [10]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute                     value    customers_name  \
0    pod_compatibility                       n/a  bedbathandbeyond   
1    pod_compatibility                       n/a  bedbathandbeyond   
2    pod_compatibility                       n/a  bedbathandbeyond   
3    pod_compatibility                       n/a  bedbathandbeyond   
4    pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond   
..                 ...                       ...               ...   
670  pod_compatibility                       n/a  bedbathandbeyond   
671  pod_compatibility                       n/a  bedbathandbeyond   
672  pod_compatibility                       n/a  bedbathandbeyond   
673  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond   
674  pod_compatibility                       n/a  bedbathandbeyond   

    external_id  \
0      68642839   
1      46946423   
2      67673377   
3      66450375   
4      66326403   
..          ...   
670    64803784   
671    64803807   
672    68287252   
673    66326045   
674    17429019   

                                                                     long_desc  \
0    Zwilling® J.A. Henckels Sorrento Plus 27 oz. French Press in Clear/Silver   
1                                Zojirushi Fresh Brew Plus 12-Cup Coffee Maker   
2                       Zojirushi™ 10-Cup Thermal Carafe Coffee Maker in Black   
3                    Zaasenhaus 34 oz. Stainless Steel Hot & Cold Brew Infuser   
4                           West Virginia University Single Serve Coffee Maker   
..                                                                         ...   
670                    ASOBU 32 oz. Pour Over Insulated Coffee Maker in Copper   
671                     ASOBU 32 oz. Pour Over Insulated Coffee Maker in Black   
672                               Aroma® 6-Cup Programable Tea Maker in Pastel   
673                         Arizona State University Single Serve Coffee Maker   
674                                         AeroPress® Coffee & Espresso Maker   

    resolution matches  
0     standard      []  
1     standard      []  
2     standard      []  
3     standard      []  
4     standard      []  
..         ...     ...  
670   standard      []  
671   standard      []  
672   standard      []  
673   standard      []  
674   standard      []  

[675 rows x 7 columns]

In [ ]:
10"x20"

In [11]:
df[(df['matches'].astype(str)=='[]')&(df['value'].astype(str)=='n/a')]

attribute value    customers_name external_id  \
0    pod_compatibility   n/a  bedbathandbeyond    68642839   
1    pod_compatibility   n/a  bedbathandbeyond    46946423   
2    pod_compatibility   n/a  bedbathandbeyond    67673377   
3    pod_compatibility   n/a  bedbathandbeyond    66450375   
5    pod_compatibility   n/a  bedbathandbeyond    61693111   
..                 ...   ...               ...         ...   
669  pod_compatibility   n/a  bedbathandbeyond    64803814   
670  pod_compatibility   n/a  bedbathandbeyond    64803784   
671  pod_compatibility   n/a  bedbathandbeyond    64803807   
672  pod_compatibility   n/a  bedbathandbeyond    68287252   
674  pod_compatibility   n/a  bedbathandbeyond    17429019   

                                                                     long_desc  \
0    Zwilling® J.A. Henckels Sorrento Plus 27 oz. French Press in Clear/Silver   
1                                Zojirushi Fresh Brew Plus 12-Cup Coffee Maker   
2                       Zojirushi™ 10-Cup Thermal Carafe Coffee Maker in Black   
3                    Zaasenhaus 34 oz. Stainless Steel Hot & Cold Brew Infuser   
5                                   West Bend Venti Hot Beverage Maker in Grey   
..                                                                         ...   
669                       ASOBU 32 oz. Pour Over Insulated Coffee Maker in Red   
670                    ASOBU 32 oz. Pour Over Insulated Coffee Maker in Copper   
671                     ASOBU 32 oz. Pour Over Insulated Coffee Maker in Black   
672                               Aroma® 6-Cup Programable Tea Maker in Pastel   
674                                         AeroPress® Coffee & Espresso Maker   

    resolution matches  
0     standard      []  
1     standard      []  
2     standard      []  
3     standard      []  
5     standard      []  
..         ...     ...  
669   standard      []  
670   standard      []  
671   standard      []  
672   standard      []  
674   standard      []  

[545 rows x 7 columns]

In [12]:
df['matches'].explode().value_counts()

Series([], Name: matches, dtype: int64)

In [13]:
df[0:50]

attribute                     value    customers_name external_id  \
0   pod_compatibility                       n/a  bedbathandbeyond    68642839   
1   pod_compatibility                       n/a  bedbathandbeyond    46946423   
2   pod_compatibility                       n/a  bedbathandbeyond    67673377   
3   pod_compatibility                       n/a  bedbathandbeyond    66450375   
4   pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326403   
5   pod_compatibility                       n/a  bedbathandbeyond    61693111   
6   pod_compatibility                       n/a  bedbathandbeyond    61693128   
7   pod_compatibility                       n/a  bedbathandbeyond    65594759   
8   pod_compatibility                       n/a  bedbathandbeyond    65667941   
9   pod_compatibility                       n/a  bedbathandbeyond    46948953   
10  pod_compatibility                       n/a  bedbathandbeyond    17550799   
11  pod_compatibility                       n/a  bedbathandbeyond    65589830   
12  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326397   
13  pod_compatibility                       n/a  bedbathandbeyond    61456235   
14  pod_compatibility                       n/a  bedbathandbeyond    66326410   
15  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326359   
16  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326342   
17  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326380   
18  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326328   
19  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326304   
20  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326281   
21  pod_compatibility                       n/a  bedbathandbeyond    66326236   
22  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326250   
23  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326229   
24  pod_compatibility                       n/a  bedbathandbeyond    66326298   
25  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326205   
26  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326175   
27  pod_compatibility  Single-Serve Coffee Cups  bedbathandbeyond    66326168   
28  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326144   
29  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326120   
30  pod_compatibility                       n/a  bedbathandbeyond    66326106   
31  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326090   
32  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326038   
33  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326021   
34  pod_compatibility                       n/a  bedbathandbeyond    43195183   
35  pod_compatibility                       n/a  bedbathandbeyond    47355392   
36  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    47355378   
37  pod_compatibility                       n/a  bedbathandbeyond    47355385   
38  pod_compatibility                       n/a  bedbathandbeyond    42857525   
39  pod_compatibility                       n/a  bedbathandbeyond    42857532   
40  pod_compatibility  Single-Serve Coffee Pods  bedbathandbeyond    66326366   
41  pod_compatibility                       n/a  bedbathandbeyond    63289435   
42  pod_compatibility                       n/a  bedbathandbeyond    68583798   
43  pod_compatibility                       n/a  bedbathandbeyond    68583781   
44  pod_compatibility                       n/a  bedbathandbeyond    68651619   
45  pod_compatibility                       n/a  bedbathandbeyond    68651626   
46  pod_compatibility                       n/a  bedbathandbeyond    68651633   
47  pod_compatibility                       n/a  bedbathandbeyond    68651602   
48  pod_compatibility                      

In [14]:
error
10"  20" 

NameError: name 'error' is not defined

In [ ]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
    data[['length', 'width']]=data['matches'].str.split(" x ",expand=True,)

In [ ]:
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))

In [ ]:
def replace_none(data):
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)
    data['width']=data['width'].apply(lambda x: 0 if x == 'None' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == 'None' else x)

In [ ]:
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")

In [15]:
def min_max(data):
    data['length']=data['length'].astype(float)
    data['width']=data['width'].astype(float)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [16]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split(' ')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
# error

In [ ]:
# x=df['matches']
# x

In [ ]:
# x[0:50]

In [ ]:
# x.str.split(",",expand=True,)

In [ ]:
# x=df
# x['matches']=x['matches'].str.get(0).apply(lambda x:x.lower())
# x

In [ ]:
# x[['length', 'width']]=x['matches'].str.split(",",expand=True,)

In [ ]:
# error

In [ ]:
split_col(df) #might throw an error and have to drop nane

In [ ]:
df['width']=df['width'].astype(str).apply(lambda x: 0 if x == 'nan' else x)
df

In [ ]:
df['length']=df['length'].astype(str)
df['width']=df['width'].astype(str)
df['length']=df['length'].apply(lambda quote: quote.replace('-', ' '))
df['width']=df['width'].apply(lambda quote: quote.replace('-', ' '))
df['length']=df['length'].apply(lambda quote: quote.replace('inch', ''))
df['width']=df['width'].apply(lambda quote: quote.replace('inch', ''))
df

In [ ]:
df['length']=df['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df['width']=df['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df

In [ ]:
a=df['length'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['length']=c
df

In [ ]:
a=df['width'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['width']=c
df

In [ ]:
# x=df
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# x['new'] = c
# x

In [ ]:
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)

In [ ]:
# remove_fractions(df['length'])

In [ ]:
# x=df
# a=eval(x['length'][3:4].astype(str))
# # numbers = eval(a)

In [ ]:
# float(sum(Fraction(s) for s in a))

In [ ]:
# from fractions import Fraction
# float(sum(Fraction(s) for s in '  12 1/2'.split()))

In [ ]:
# x=df
# x['length'] = x['length'].apply(lambda quote: quote.replace('-', ''))
# x['length'] = x['length'].apply(lambda quote: quote.replace('inch', ''))
# # x['length'] = x['length'].apply(lambda x: x.str)
# x

In [ ]:
# x[['first', 'second']]=x['length'].str.split("/",expand=True,)
# x[['third', 'fourth']= x['first'].split()
# result = x['third'] + (float(x['first'])/float(x['second']))
# result

In [ ]:
# num,den = s.split( '/' )
# wh, num = num.split()
# result = wh + (float(num)/float(den))

In [ ]:
# a=x['length'][3:4]
# a.str.split(' ')
# b= [[" ".join(tag.split(" ")[:2]) , " ".join(tag.split(" ")[2:])] for tag in a][0]
# b

In [ ]:
# c=b[1]
# c.split('/')

In [ ]:
# b[0]
# b

In [ ]:
# a.apply(lambda x: sum(Fraction(x.split('/'))))

In [ ]:
# float(sum(Fraction(s) for s in x['length'].split()))
# x['length']=x['length'].apply(lambda a: float(sum(Fraction(a.split()))))

In [ ]:
# for i in x['length']:
#     if x['length'][i].find('/') > -1:
#         float(sum(Fraction(s) for s in x['length'][i].split()))
#     else:
#         x['length']

In [ ]:
# float(sum(Fraction(s) for s in x['length'][i].split()))
# .apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))

In [ ]:
# def fraction_combo(numbers):
#     if numbers.find('/') > -1:
#         numbers.apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))
#         #numbers = literal_eval(numbers)
#         #inte, fract = numbers.split(' ')
#         #nom, den = fract.split('/')
#         #tmp = float(nom)/float(den)
#         #num = int(inte) + tmp
#         return str(numbers)
#     else:
#         return numbers

In [ ]:
# a=x['length']

In [ ]:
# remove_fractions(df)

In [ ]:
# x['length'] = x['length'].apply(lambda x: remove_fractions(x))
# x.head

In [ ]:
# x['length'] = x['length'].apply(lambda x: x.split(' '))
# x.head()

In [ ]:
min_max(df) #error base 10 when there is only a value for max and not min

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
# # range function builder
# string = range_string.replace(metric, '')
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

In [ ]:
df

In [ ]:
matches=df
matches['Max']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
matches

In [ ]:
matches[curation_col] = matches['Max'].astype(str)+' in'
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)